<a href="https://colab.research.google.com/github/jeffersonramelo/Textual-Analysis/blob/main/C2_sentimento_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Caminho do arquivo CSV
caminho_arquivo = "/content/C123_dataset sentimento rotulados.csv"

# Ler o arquivo CSV
import pandas as pd
df = pd.read_csv(caminho_arquivo, sep=",", encoding='latin-1')

In [25]:
#vizualizar dados
df.head()

,sentimento_rotulado,texto,id
0,neutral,"According to Gran , the company has no plans t...",1
1,neutral,Technopolis plans to develop in stages an area...,2
2,negative,The international electronic industry company ...,3
3,positive,With the new production plant the company woul...,4
4,positive,According to the company 's updated strategy f...,5


In [26]:
#quantos dados positivo e negativos?
df.sentimento_rotulado.value_counts()

neutral     2879
positive    1363
negative     604
Name: sentimento_rotulado, dtype: int64

In [27]:
# Acessar os textos contidos nas celulas, só substituir o número da celula que deseja vizualizar
celula = df.loc[4, 'texto']

# Imprimir o conteúdo da célula
print(celula)


According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .


pre-processamento do texto (stop words, misnusculas, caracteres especiais, stemização e tokenização)

baixar stop words do pacote NLTK, tem disponibilidade de vários idiomas, no português pode seguir o comando: stopwords_pt = stopwords.words('portuguese')

In [28]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
stopwords_en

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [30]:
import re

def preprocess_text(text):
    # Converter para minúsculas
    text = text.lower()

    # Remover números
    text = re.sub(r'\d+', '', text)

    # Remover sinais de pontuação e caracteres especiais
    text = re.sub(r'[^\w\s]', '', text)

    # Remover stopwords
    stopwords_en = stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in stopwords_en])

    return text

df['texto'] = df['texto'].apply(preprocess_text)

In [31]:
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# Criar uma instância do stemmer
stemmer = PorterStemmer()

# Função para realizar a stemização das palavras em um texto
def stem_text(text):
    # Tokenizar o texto em palavras
    words = word_tokenize(text)

    # Aplicar a stemização em cada palavra
    stemmed_words = [stemmer.stem(word) for word in words]

    # Juntar as palavras novamente em um texto
    stemmed_text = ' '.join(stemmed_words)

    return stemmed_text

# Aplicar a função de stemização na coluna "texto" do DataFrame
df['texto'] = df['texto'].apply(stem_text)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [32]:
df

,sentimento_rotulado,texto,id
0,neutral,accord gran compani plan move product russia a...,1
1,neutral,technopoli plan develop stage area less squar ...,2
2,negative,intern electron industri compani elcoteq laid ...,3
3,positive,new product plant compani would increas capac ...,4
4,positive,accord compani updat strategi year baswar targ...,5
...,...,...,...
4841,negative,london marketwatch share price end lower londo...,4842
4842,neutral,rinkuskiai beer sale fell per cent million lit...,4843
4843,negative,oper profit fell eur mn eur mn includ vessel s...,4844
4844,negative,net sale paper segment decreas eur mn second q...,4845


Naive Bayes

In [33]:
pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [43]:
# Dividir o DataFrame em dados de treinamento e teste
X = df['texto']
y = df['sentimento_rotulado']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preencher valores nulos com uma string vazia
imputer = SimpleImputer(strategy='constant', fill_value='')
X_train = pd.DataFrame(X_train, columns=['texto'])
X_test = pd.DataFrame(X_test, columns=['texto'])
X_train['texto'] = imputer.fit_transform(X_train[['texto']])
X_test['texto'] = imputer.transform(X_test[['texto']])

# Pré-processamento dos dados de texto
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train['texto'])
X_test = vectorizer.transform(X_test['texto'])

# Inicializar os classificadores
mnb = MultinomialNB()
gnb = GaussianNB()
bnb = BernoulliNB()


# Realizar validação cruzada nos classificadores
cv_scores_mnb = cross_val_score(mnb, X_train, y_train, cv=5)
cv_scores_gnb = cross_val_score(gnb, X_train.toarray(), y_train, cv=5)
cv_scores_bnb = cross_val_score(bnb, X_train, y_train, cv=5)




# Treinar os classificadores nos dados de treinamento completos
mnb.fit(X_train, y_train)
gnb.fit(X_train.toarray(), y_train)
bnb.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_mnb = mnb.predict(X_test)
y_pred_gnb = gnb.predict(X_test.toarray())
y_pred_bnb = bnb.predict(X_test)

In [45]:
# Calcular a matriz de confusão
labels = ["negative", "neutral", "positive"]

confusion_mnb = confusion_matrix(y_test, y_pred_mnb, labels=labels)
print("Matriz de Confusão - Multinomial Naive Bayes:")
print(pd.DataFrame(confusion_mnb, index=labels, columns=labels))
print()

confusion_gnb = confusion_matrix(y_test, y_pred_gnb, labels=labels)
print("Matriz de Confusão - Gaussian Naive Bayes:")
print(pd.DataFrame(confusion_gnb, index=labels, columns=labels))
print()

confusion_bnb = confusion_matrix(y_test, y_pred_bnb, labels=labels)
print("Matriz de Confusão - Bernoulli Naive Bayes:")
print(pd.DataFrame(confusion_bnb, index=labels, columns=labels))
print()

Matriz de Confusão - Multinomial Naive Bayes:
          negative  neutral  positive
negative        59       34        17
neutral         14      489        68
positive        18      124       147

Matriz de Confusão - Gaussian Naive Bayes:
          negative  neutral  positive
negative        64       28        18
neutral         82      305       184
positive        72      107       110

Matriz de Confusão - Bernoulli Naive Bayes:
          negative  neutral  positive
negative        16       55        39
neutral          1      540        30
positive         4      146       139



In [44]:
# Calcular os indicadores de desempenho
accuracy_mnb = accuracy_score(y_test, y_pred_mnb)
precision_mnb = precision_score(y_test, y_pred_mnb, average='weighted')
recall_mnb = recall_score(y_test, y_pred_mnb, average='weighted')
f1_mnb = f1_score(y_test, y_pred_mnb, average='weighted')

accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
precision_gnb = precision_score(y_test, y_pred_gnb, average='weighted')
recall_gnb = recall_score(y_test, y_pred_gnb, average='weighted')
f1_gnb = f1_score(y_test, y_pred_gnb, average='weighted')

accuracy_bnb = accuracy_score(y_test, y_pred_bnb)
precision_bnb = precision_score(y_test, y_pred_bnb, average='weighted')
recall_bnb = recall_score(y_test, y_pred_bnb, average='weighted')
f1_bnb = f1_score(y_test, y_pred_bnb, average='weighted')

# Exibir os indicadores de desempenho
print("Indicadores de Desempenho:")
df_results = pd.DataFrame({
    'Modelo': ['Multinomial Naive Bayes', 'Gaussian Naive Bayes', 'Bernoulli Naive Bayes'],
    'Acurácia': [accuracy_mnb, accuracy_gnb, accuracy_bnb],
    'Precisão': [precision_mnb, precision_gnb, precision_bnb],
    'Recall': [recall_mnb, recall_gnb, recall_bnb],
    'F1-score': [f1_mnb, f1_gnb, f1_bnb]
})
print(df_results)


Indicadores de Desempenho:
                    Modelo  Acurácia  Precisão    Recall  F1-score
0  Multinomial Naive Bayes  0.716495  0.707211  0.716495  0.707368
1     Gaussian Naive Bayes  0.493814  0.546383  0.493814  0.508492
2    Bernoulli Naive Bayes  0.716495  0.714487  0.716495  0.678922


Suporte Vector Machine: procura uma reta que melhor separa as classes

In [38]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Dividir o DataFrame em dados de treinamento e teste
X = df['texto']
y = df['sentimento_rotulado']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preencher valores nulos com uma string vazia
imputer = SimpleImputer(strategy='constant', fill_value='')
X_train = pd.DataFrame(X_train, columns=['texto'])
X_test = pd.DataFrame(X_test, columns=['texto'])
X_train['texto'] = imputer.fit_transform(X_train[['texto']])
X_test['texto'] = imputer.transform(X_test[['texto']])

# Pré-processamento dos dados de texto
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train['texto'])
X_test = vectorizer.transform(X_test['texto'])

# Inicializar o classificador SVM
svm = SVC()

# Realizar validação cruzada no classificador SVM
cv_scores_svm = cross_val_score(svm, X_train, y_train, cv=5)

# Treinar o classificador SVM nos dados de treinamento completos
svm.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_svm = svm.predict(X_test)

# Calcular a matriz de confusão
labels = ["negative", "neutral", "positive"]
confusion_svm = confusion_matrix(y_test, y_pred_svm, labels=labels)
print("Matriz de Confusão - SVM:")
print(pd.DataFrame(confusion_svm, index=labels, columns=labels))
print()

# Calcular os indicadores de desempenho
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm, average='weighted')
recall_svm = recall_score(y_test, y_pred_svm, average='weighted')
f1_svm = f1_score(y_test, y_pred_svm, average='weighted')

# Exibir os indicadores de desempenho
print("Indicadores de Desempenho:")
df_results_svm = pd.DataFrame({
    'Modelo': ['SVM'],
    'Acurácia': [accuracy_svm],
    'Precisão': [precision_svm],
    'Recall': [recall_svm],
    'F1-score': [f1_svm]
})
print(df_results_svm)


Matriz de Confusão - SVM:
          negative  neutral  positive
negative        39       56        15
neutral          3      557        11
positive        14      169       106

Indicadores de Desempenho:
  Modelo  Acurácia  Precisão    Recall  F1-score
0    SVM  0.723711  0.737518  0.723711  0.687992


Redes Neurais

A rede neural utilizada no exemplo é um Perceptron de Múltiplas Camadas (Multilayer Perceptron, MLP). O MLP é uma arquitetura de rede neural artificial que consiste em várias camadas de neurônios, incluindo uma camada de entrada, uma ou mais camadas ocultas e uma camada de saída. Cada neurônio em uma camada está conectado a todos os neurônios na camada seguinte, formando uma rede densamente conectada.

In [40]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd

# Dividir o DataFrame em dados de treinamento e teste
X = df['texto']
y = df['sentimento_rotulado']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pré-processamento dos dados de texto
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Inicializar o classificador MLP
mlp = MLPClassifier(hidden_layer_sizes=(100,), random_state=42)

# Treinar o classificador MLP
mlp.fit(X_train, y_train)

# Fazer previsões nos dados de teste
y_pred_mlp = mlp.predict(X_test)

# Gerar a matriz de confusão
labels = ["negative", "neutral", "positive"]
confusion_mtx_mlp = confusion_matrix(y_test, y_pred_mlp, labels=labels)

print("Matriz de Confusão - MLP:")
print(pd.DataFrame(confusion_mtx_mlp, index=labels, columns=labels))

# Calcular os indicadores de desempenho
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
precision_mlp = precision_score(y_test, y_pred_mlp, average=None, labels=labels)
recall_mlp = recall_score(y_test, y_pred_mlp, average=None, labels=labels)
f1_mlp = f1_score(y_test, y_pred_mlp, average=None, labels=labels)

print("Acurácia - MLP:", accuracy_mlp)
print("Precisão - MLP:", precision_mlp)
print("Recall - MLP:", recall_mlp)
print("F1-score - MLP:", f1_mlp)



Matriz de Confusão - MLP:
          negative  neutral  positive
negative        61       30        19
neutral         27      460        84
positive        16      109       164
Acurácia - MLP: 0.7061855670103093
Precisão - MLP: [0.58653846 0.76794658 0.61423221]
Recall - MLP: [0.55454545 0.8056042  0.56747405]
F1-score - MLP: [0.57009346 0.78632479 0.58992806]
